In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
from keras.utils import np_utils
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

data_dir = os.path.join(os.getcwd(),"slides_ds")

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(300,300),
  batch_size=32)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(300,300),
  batch_size=32)

normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)


Found 284 files belonging to 2 classes.
Using 228 files for training.
Found 284 files belonging to 2 classes.
Using 56 files for validation.
0.0 1.0
Epoch 1/3
8/8 [==============================] - 50s 4s/step - loss: 1.4860 - accuracy: 0.6228 - val_loss: 0.5843 - val_accuracy: 0.6429
Epoch 2/3
8/8 [==============================] - 27s 3s/step - loss: 0.5003 - accuracy: 0.7719 - val_loss: 0.4276 - val_accuracy: 0.8036
Epoch 3/3
8/8 [==============================] - 17s 2s/step - loss: 0.3879 - accuracy: 0.8202 - val_loss: 0.4697 - val_accuracy: 0.7500
